In [12]:
# i del 
def is_strong_odd_independent_set(G, S):
    """
    Preveri, ali je S odd independent množica v grafu G.
    
    Args:
        G: Graf
        S: Množica vozlišč
    
    Returns:
        bool: True če je S odd independent, False sicer
    """
    S_set = set(S)
    
    # Preveri neodvisnost
    for u in S_set:
        for v in G.neighbors(u):
            if v in S_set:
                return False
    
    # Preveri strong odd pogoj
    for u in G.vertices():
        if u not in S_set:
            count = sum(1 for v in G.neighbors(u) if v in S_set)
            # Mora biti liho
            if count % 2 == 0:
                return False
    
    return True

def satisfies_condition_i_strict(G):

    if not G.is_connected():
        return False
    
    degrees = G.degree()
    if len(set(degrees)) != 1:
        return False
    
    r = degrees[0]
    
    if r % 2 == 0:
        return False
    
    if G.diameter() > 3:
        return False
    
    # izračun α_od z ILP
    alpha = alpha_od_ilp_correct(G)
    
    # za vsakega preveri
    for v in G.vertices():
        Nv = G.neighbors(v)
        
        if not is_strong_odd_independent_set(G, Nv):
            return False
        
        if len(Nv) != alpha:
            return False
    
    return True


In [13]:
# ii del
def is_strong_odd_independent_set(G, S):
    """
    Preveri, ali je S odd independent množica v grafu G.
    
    Args:
        G: Graf
        S: Množica vozlišč
    
    Returns:
        bool: True če je S odd independent, False sicer
    """
    S_set = set(S)
    
    # Preveri neodvisnost
    for u in S_set:
        for v in G.neighbors(u):
            if v in S_set:
                return False
    
    # Preveri strong odd pogoj
    for u in G.vertices():
        if u not in S_set:
            count = sum(1 for v in G.neighbors(u) if v in S_set)
            # Mora biti liho
            if count % 2 == 0:
                return False
    
    return True

def satisfies_condition_ii(G):

    if not G.is_connected():
        return False
    
    degrees = G.degree()
    if len(set(degrees)) != 1:
        return False
    
    r = degrees[0]
    
    # r mora biti liho
    if r % 2 == 0:
        return False
    
    # brez trikotnikov
    if G.clique_number() >= 3:
        return False
    
    # nujni pogoj iz naloge
    if G.diameter() > 3:
        return False
    
    # izračun α_od
    alpha = alpha_od_ilp_correct(G)
    
    # zdaj obstoj vsaj enega v
    for v in G.vertices():
        Nv = G.neighbors(v)
        
        if is_strong_odd_independent_set(G, Nv) and len(Nv) == alpha:
            return True
    
    return False


In [14]:

def satisfies_condition_iii(G, r):
    return alpha_od_ilp_correct(G) == r

In [19]:
def check_conditions(G, r):
    results = {
        "i": satisfies_condition_i_strict(G),
        "ii": satisfies_condition_ii(G),
        "iii": satisfies_condition_iii(G, r)
    }
    return results

In [20]:
G = graphs.PetersenGraph()

print("SRG parametri:", G.is_strongly_regular(parameters=True))
print(check_conditions(G, 3))

SRG parametri: (10, 3, 0, 1)
{'i': True, 'ii': True, 'iii': True}


In [21]:
G = graphs.ClebschGraph()

print("SRG parametri:", G.is_strongly_regular(parameters=True))
print(check_conditions(G, 5))

SRG parametri: (16, 5, 0, 2)
{'i': False, 'ii': False, 'iii': False}


In [22]:
for r in [3,5,7]:
    G = graphs.CompleteBipartiteGraph(r, r)

    print(f"\nK_{r},{r}")
    print("SRG parametri:", G.is_strongly_regular(parameters=True))
    print("Pogoji:", check_conditions(G, r))


K_3,3
SRG parametri: (6, 3, 0, 3)
Pogoji: {'i': True, 'ii': True, 'iii': True}

K_5,5
SRG parametri: (10, 5, 0, 5)
Pogoji: {'i': True, 'ii': True, 'iii': True}

K_7,7
SRG parametri: (14, 7, 0, 7)
Pogoji: {'i': True, 'ii': True, 'iii': True}


In [37]:
import os

targets = [
    (True, True, True),
    (False, True, True),
    (False, False, True)
]

limits = {
    (True, True, True): 3,
    (False, True, True): 1,
    (False, False, True): 3
}

# ustvarimo mape
for key in targets:
    folder = f"graphs_{key[0]}_{key[1]}_{key[2]}"
    os.makedirs(folder, exist_ok=True)

# odpremo datoteke
files = {}
for key in targets:
    filename = f"graphs_{key[0]}_{key[1]}_{key[2]}.txt"
    files[key] = open(filename, "w")

counts = {key: 0 for key in targets}

def all_done():
    return all(counts[k] >= limits[k] for k in targets)

for r in [3,5,7]:

    print(f"\n===== r = {r} =====")

    for n in range(r+1, 17):

        if n % 2 != 0:
            continue

        for i, G in enumerate(graphs.nauty_geng(f"{n} -d{r} -D{r}")):

            res = check_conditions(G, r)
            key = (res["i"], res["ii"], res["iii"])

            if key in targets and counts[key] < limits[key]:

                counts[key] += 1
                code = G.graph6_string()

                # zapis v txt
                files[key].write(
                    f"n={n}, r={r}, kombinacija={key}, graph6={code}\n"
                )

                # shrani sliko
                folder = f"graphs_{key[0]}_{key[1]}_{key[2]}"
                image_name = f"{folder}/n{n}_r{r}_ex{counts[key]}.png"
                G.plot().save(image_name)

                print(f"Shranjeno: {key}, primer {counts[key]}")

                # če so vse kvote zapolnjene → ustavi vse
                if all_done():
                    break

        if all_done():
            break

    if all_done():
        break

# zapremo datoteke
for f in files.values():
    f.close()

print("\nPOVZETEK:")
for key in targets:
    print(f"{key}: {counts[key]} / {limits[key]}")


===== r = 3 =====


Shranjeno: (True, True, True), primer 1


Shranjeno: (False, False, True), primer 1


Shranjeno: (False, False, True), primer 2
Shranjeno: (False, False, True), primer 3


Shranjeno: (False, True, True), primer 1
Shranjeno: (True, True, True), primer 2



===== r = 5 =====
Shranjeno: (True, True, True), primer 3

POVZETEK:
(True, True, True): 3 / 3
(False, True, True): 1 / 1
(False, False, True): 3 / 3
